In [1]:
import numpy as np
import glob
import copy
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
from sklearn.metrics import accuracy_score
import copy
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader, TensorDataset
from imblearn.over_sampling import SMOTE
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import f1_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Load Test Data

In [2]:
def load_data():
    path = f'./python'
    files = glob.glob(path + '/test*.npy')
    files = sorted(files)
    
    result = []
    labels = [file.split('/')[-1].replace('.npy', '') for file in files]
    
    for i, file in enumerate(files):
        print(f"Loaded {i + 1}/{len(files)} test files.", end="\r")
        data = np.load(file)
        result.append(data)
        
    result = np.array(result)
    
    return result, labels

In [3]:
test_data, labels = load_data()

In [4]:
scaler = StandardScaler()
test_data = np.array([scaler.fit_transform(arr) for arr in test_data])

In [6]:
test_data.shape

(16, 3000, 548)

# Load Models

In [7]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

### Neural networks

In [8]:
class BinaryClassifier(nn.Module):
    def __init__(self, input_dim):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(p=0.5)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))  
        x = self.dropout(x)      
        
        x = F.relu(self.fc2(x))  
        x = self.dropout(x)      
        
        x = F.relu(self.fc3(x)) 
        x = self.dropout(x)      
        
        x = F.relu(self.fc4(x))  
        x = self.dropout(x)      
        
        x = torch.sigmoid(self.fc5(x))  
        return x

In [9]:
class BirdClassifier(nn.Module):
    def __init__(self):
        super(BirdClassifier, self).__init__()
        self.conv1 = nn.Conv1d(1, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm1d(64) 
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(128)  
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm1d(256) 
        self.pool = nn.MaxPool1d(2, 2)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(4096, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 6)  # 6 output classes

    def forward(self, x):
        x = x.unsqueeze(1)  
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  
        x = self.pool(F.relu(self.bn2(self.conv2(x)))) 
        x = self.pool(F.relu(self.bn3(self.conv3(x)))) 
        x = x.view(x.size(0), -1)
        x = F.dropout(F.relu(self.fc1(x)), p=0.2, training=self.training) 
        x = F.dropout(F.relu(self.fc2(x)), p=0.2, training=self.training) 
        x = self.fc3(x)
        return x

In [10]:
model1 = BinaryClassifier(input_dim=548).to(device)
model1.load_state_dict(torch.load('model1.pth'))

<All keys matched successfully>

In [11]:
model2 = BirdClassifier().to(device)
model2.load_state_dict(torch.load('model2.pth'))

<All keys matched successfully>

# Evaluate Data

In [12]:
# Assuming X_unlabeled is a numpy array
X_unlabeled = torch.from_numpy(test_data).float().to(device)
batch_size = 64

# Create a DataLoader for the unlabeled data
unlabeled_dataset = TensorDataset(X_unlabeled)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=False)


# Prediction function
def predict(model, unlabeled_loader):
    model.eval()  # Ensure the model is in evaluation mode
    result = []
    predictions = []

    with torch.no_grad():
        for sample in test_data:
            X_unlabeled = torch.from_numpy(sample).float().to(device)
            # Create a DataLoader for the unlabeled data
            unlabeled_dataset = TensorDataset(X_unlabeled)
            unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=False)
            
            for inputs in unlabeled_loader:
                inputs = inputs[0].to(device)
                outputs = model(inputs)  # Pass data through model
                predicted = (outputs > 0.5).float() # Get the predicted classes
                predictions.extend(predicted.cpu().numpy().tolist())  # Add the predictions to our list
            
            result.append(predictions)
            predictions = []

    return np.array(result)

# Predict
predictions = predict(model1, unlabeled_loader).astype(int)

In [13]:
sub = ""

for i, prediction in enumerate(predictions):
    y_pred = prediction.astype(str).flatten()
    
    y_str = ','.join(y_pred)
    result = f'{labels[i]},{y_str}\n'
    sub += result
    
sub

'test00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,1,1,0,0,0,1,1,1,1,0,0,0,0,1,0,1,1,0,0,0,1,1,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,1,1,1,0,0,0,0,1,0,1,1,0,0,0,1,1,1,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,1,

In [14]:
predictions.shape

(16, 3000, 1)

In [15]:
indices = np.array([  0,   3,   4,   6,   8,   9,  11,  12,  13,  15,  16,  17,  18,
         19,  20,  21,  22,  23,  24,  25,  27,  28,  29,  30,  31,  32,
         33,  34,  35,  36,  37,  38,  39,  40,  46,  47,  69,  70,  71,
         72,  73,  74,  75,  79,  80,  81,  82,  83,  84,  85,  87,  95,
        109, 127, 130, 137, 138, 139, 141, 142, 143, 144, 145, 146, 147,
        148, 149, 150, 151, 152, 153, 154, 155, 156, 236, 256, 258, 268,
        270, 271, 272, 273, 274, 278, 285, 286, 287, 288, 289, 290, 292,
        293, 294, 295, 296, 297, 305, 306, 345, 346, 347, 348, 349, 350,
        351, 361, 372, 404, 410, 411, 412, 413, 414, 415, 416, 417, 418,
        419, 420, 422, 462, 502, 522, 536, 537, 538, 539, 544])

In [16]:
test_data = test_data[:, :, indices]
test_data.shape

(16, 3000, 128)

In [38]:
a = np.squeeze(predictions)
test_data_model2 = test_data[a == 1]
test_data_model2.shape

X_test = torch.from_numpy(test_data_model2).float().to(device)

In [39]:
with torch.no_grad():
        outputs = model2(X_test)  # Forward pass through the model
        _, predicted = torch.max(outputs, 1)

In [40]:
bird_sounds = predicted.cpu().numpy()

In [41]:
bird_sounds += 1
bird_sounds.tolist()

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 4,
 4,
 2,
 4,
 4,
 4,
 4,
 4,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 5,
 6,
 5,
 5,
 6,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 1,
 6,
 6,
 6,
 5,
 6,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 4,
 2,
 4,
 4,
 2,
 2,
 2,
 2,
 2,
 4,
 2,
 2,
 4,
 4,
 4,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 4,
 2,
 2,
 2,
 2,
 4,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 5,
 5,
 6,
 6,
 6,
 6,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 5,
 1,
 1,
 1,
 1,
 6,
 6,
 6,
 6,
 6,
 1,
 4,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 4,
 4,
 3,
 3,
 4,
 6,
 4,
 4,
 2,
 4,
 4,
 2,
 4,
 4,
 4,
 3,
 4,
 2,
 2,
 2,
 2,
 6,
 3,
 3,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,


In [48]:
predictions[a == 1] = bird_sounds.reshape(-1, 1)

In [50]:
np.unique(predictions)

array([0, 1, 2, 3, 4, 5, 6])

In [51]:
sub = ""

for i, prediction in enumerate(predictions):
    y_pred = prediction.astype(str).flatten()
    
    y_str = ','.join(y_pred)
    result = f'{labels[i]},{y_str}\n'
    sub += result
    
sub

'test00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,1,1,0,0,0,1,1,1,1,0,0,0,0,1,0,1,1,0,0,0,1,2,1,1,0,0,0,0,5,0,0,0,0,0,5,0,0,0,5,0,0,0,5,0,0,5,0,0,5,0,0,0,0,0,0,5,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,1,1,1,0,0,0,0,1,0,2,2,0,0,0,1,1,1,1,0,0,0,2,1,0,0,5,0,0,0,0,0,5,0,0,5,0,0,5,0,0,5,0,0,5,0,0,5,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,4,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,6,6,6,6,6,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,3,

In [52]:
with open ('./submission_hierarchical.csv', 'w') as file:
    file.write(sub)